In [1]:
import math
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# i_star

In [10]:
def i_star(beta_prime1, stagger, a_c, theta, t_max_c, sigma_r, blade_type):
    
    beta1_calculated = beta_prime1
    beta1 = 0
    
    while abs(beta1-beta1_calculated)>0.001:
        
        beta1 = beta1_calculated
        
        #k_sh:
        k_sh_dic = {
            'C_Series': 1.1,
            '65_Series': 1,
            'DCA': 0.7,
        }
    
        k_sh = k_sh_dic[blade_type[0]]
    
        #k_ti:
        q = 0.28/(0.1+t_max_c**0.3)
        k_ti = (10*t_max_c)**q
    
        #circular:
        e = 0.65-0.002*theta
        sigma = sigma_r
        alpha_star_circular = (3.6*k_sh*k_ti+0.3532*theta*(a_c**0.25))*(sigma**e)
        n = 0.025*sigma-0.06-((beta1/90)**(1+1.2*sigma))/(1.5+0.43*sigma)
        p = 0.914+sigma**3/160
        i_star_0_10 = (beta1**p)/(5+46*e**(-2.3*sigma))-0.1*(sigma**3)*(e**((beta1-70)/4))
        i_star_circular = alpha_star_circular+stagger-beta_prime1
        #i_star_circular = k_sh*k_ti*i_star_0_10+n*theta
    
        #parabolic:
        beta_prime_bar1 = abs(stagger)+0.5*theta
        alpha_star_parabolic = beta_prime_bar1-abs(stagger)+(a_c-0.5)*theta
        i_star_parabolic = alpha_star_parabolic+stagger-beta_prime1
    
        i_star_dic = {
            'circular'  : i_star_circular,
            'parabolic' : i_star_parabolic,
        }

        alpha_star_dic = {
            'circular'  : alpha_star_circular,
            'parabolic' : alpha_star_parabolic,
        }
    
        i_star = i_star_dic[blade_type[1]]
        alpha_star = alpha_star_dic[blade_type[1]]
        
        beta1_calculated = i_star+beta_prime1
    
    
    return {'i_star': i_star, 'alpha_star': alpha_star, 'beta1_star': beta1, 'E-Check': abs(beta1-beta1_calculated)}

# i_c

In [32]:
def i_c(i_star, alpha_star, beta1_star, theta, stagger):
    
    beta1_c_calculated = beta1_star-20
    beta1_c = 0
    
    while abs(beta1_c_calculated-beta1_c)>0.001:
        
        beta1_c = beta1_c_calculated
        i_c = i_star-9+(1-(30/beta1_c)**0.48)*(theta/4.176)
        alpha_c  = i_c-i_star+alpha_star
        beta1_c_calculated = alpha_c+stagger
        
    #1 degree less:
    i_c = i_c-1
    alpha_c  = i_c-i_star+alpha_star
    beta1_c = alpha_c+stagger
    
        
    return {'i_c': i_c, 'alpha_c': alpha_c, 'beta1_c': beta1_c, 'E-Check': abs(beta1_c-beta1_c_calculated)}
    

# i_s

In [7]:
def i_s(i_star, alpha_star, beta1_star, theta, stagger):
    
    beta1_s_calculated = beta1_star+20
    beta1_s = 0
    
    while abs(beta1_s_calculated-beta1_s)>0.001:
        
        beta1_s = beta1_s_calculated
        i_s = i_star+10.3+(2.92-(beta1_s/15.6))*(theta/8.2)
        alpha_s  = i_s-i_star+alpha_star
        beta1_s_calculated = alpha_s+stagger
        
    return {'i_s': i_s, 'alpha_s': alpha_s, 'beta1_s': beta1_s, 'E-Check': abs(beta1_s-beta1_s_calculated)}
    

# Thermodynamic Properties

In [90]:
def properties(data):
    
    #assuming that We have calorically perfect gas:

    P_ref = 1                               #bar
    T_ref = 298.15                          #kelvin
    h_ref = 302.9068030977254               #kJ/kg
    s_ref = 5.897370050824122               #kJ/kg-k
    
    #for compressor:
    C_p = 1.004                             #kJ/kg-k
    gama = 1.4  
    R = 0.287                               #kJ/kg-k
    
    #Scenario1: when P and T are given:
    
    if ('T' in data) & ('P' in data):
        T = data['T']
        P = data['P']
        
        h = C_p*(T-T_ref)+h_ref
        s = s_ref+C_p*math.log(T/T_ref)-R*math.log(P/P_ref)
        
        return {'P[bar]':P, 'T[k]':T, 'h[kj/kg]':h, 's[kj/kg-k]':s}
        
    #Scenario2: when P and s are given:
        
    if ('P' in data) & ('s' in data):
        P = data['P']
        s = data['s']
        T = T_ref*(math.e)**(((s-s_ref)+R*math.log(P/P_ref))/C_p)
        dic = properties({'P':P , 'T':T})
        h = dic['h[kj/kg]']
        return {'P[bar]':P, 'T[k]':T, 'h[kj/kg]':h, 's[kj/kg-k]':data['s']}
    
    #Scenario3: When P and h are given:
    
    if ('P' in data) & ('h' in data):
        P = data['P']
        h = data['h']
        T = (h-h_ref)/C_p+T_ref
        dic = properties({'P':P , 'T':T})
        s = dic['s[kj/kg-k]']
        return {'P[bar]':P, 'T[k]':T, 'h[kj/kg]':data['h'], 's[kj/kg-k]':s}
    
    #Scenario4: When h and s are given:
    
    if ('h' in data) & ('s' in data):
        h = data['h']
        s = data['s']
        T = (h-h_ref)/C_p+T_ref
        P = P_ref*(math.e)**((s_ref+C_p*math.log(T/T_ref)-s)/R)
       
        return {'P[bar]':P, 'T[k]':T, 'h[kj/kg]':data['h'], 's[kj/kg-k]':data['s']}
    
        
    return {}


# Rotor Inlet

In [119]:
def rotor_inlet(r_h1, r_t1, N, beta1, alpha1, P01, T01, R):
    
    r_m = (r_h1+r_t1)/2
    U1 = 2*math.pi*r_m*N/60        #meter/sec
    
    C_m1 = U1/(math.tan(math.radians(beta1))+math.tan(math.radians(alpha1)))
    C1 = C_m1/math.cos(math.radians(alpha1))
    
    #Point01:
    Properties01 = properties({'P':P01, 'T':T01}) 
    h01 = Properties01['h[kj/kg]']   
    
    #Point1:
    h1 = h01-(C1**2)/2000
    s1 = Properties01['s[kj/kg-k]']
    
    Properties1 = properties({'h':h1, 's':s1}) 
    
    P1 = Properties1['P[bar]']
    T1 = Properties1['T[k]']
    rho1 = P1*101.325/(R*T1)
    
    W1 = C_m1/math.cos(math.radians(beta1))    
    
    #Point01_rel:
    h01_rel = h1+(W1**2)/2000
    s01_rel = s1
    
    Properties01_rel = properties({'h':h01_rel, 's':s01_rel})  
    
    P01_rel = Properties01_rel['P[bar]']    
    
    H = r_t1-r_h1
    A1 = 2*math.pi*r_m*H
    
    m_dot = rho1*A1*C_m1          #kg/sec    
    
    Properties = {
        'Point 01'     : Properties01,
        'Point 1'      : Properties1,
        'Point 01_rel' : Properties01_rel,
    }
    
    df1 = pd.DataFrame(Properties).T
    
    df2 = pd.DataFrame({
        'm_dot[kg/sec]'   : m_dot,
        'rho1[kg/m^3]'     : rho1,
        'C1[meter/sec]'   : C1,
        'W1[meter/sec]'   : W1,
        'C_m1[meter/sec]' : C_m1,
        'U1[meter/sec]'   : U1,
        'rho1[kg/sec]'    : rho1,
        'A1[m^2]'         : A1,
        'alpha1[degree]'  : alpha1,
        'beta1[degree]'   : beta1,
        
    }, index = ['value'])
    
    
    return df1, df2

# delta_star

In [6]:
def delta_star(sigma_r, beta1, t_max_c, blade_type, theta, beta_prime2, a_c, C_m1, C_m2):
    sigma = sigma_r
    
    delta_0_10 = 0.01*sigma*beta1+(0.74*sigma**1.9+3*sigma)*((beta1/90)**(1.67+1.09*sigma))    
    
    x = beta1/100

    if blade_type[0]=='C_Series':
    
        m_1 = 0.249+0.074*x-0.132*x**2+0.316*x**3
    
    else:
    
        m_1 = 0.17-0.0333*x+0.333*x**2 
        
    b = 0.9625-0.17*x-0.85*x**3    
    
    m = m_1/(sigma**b)
    
    #k_tdelta:
    k_tdelta = 6.25*t_max_c+37.5*t_max_c**2    
    
    #k_sh:
    k_sh_dic = {
        'C_Series': 1.1,
        '65_Series': 1,
        'DCA': 0.7,
    }
    
    k_sh = k_sh_dic[blade_type[0]]
    
    delta_star = k_sh*k_tdelta*delta_0_10+m*theta    
    
    #howel:
    beta2_calculated = beta_prime2+10
    beta2 = 0
    
    while abs(beta2_calculated-beta2)>0.0001:
        beta2 = beta2_calculated
        delta_star_howel = (0.92*a_c**2+0.002*beta2)*theta/(sigma**0.5-0.002*theta)+(k_sh*k_tdelta-1)*delta_0_10
        beta2_calculated = delta_star_howel+beta_prime2

    
    #delta_C_m:
    delta_C_m = 10*(1-C_m2/C_m1)    
    #delta_C_m = 0
    
    if delta_C_m>5:
        delta_C_m = 5

    if delta_C_m<(-5):
        delta_C_m = -5
        
        
    #delta_3d:
    delta_3d = 0    
    
    delta_m = delta_star+delta_C_m+delta_3d 
    delta_m_howel = delta_star_howel+delta_C_m+delta_3d 
    beta2_howel = delta_m_howel+beta_prime2
    
    return {'delta_m_liblin': delta_m, 'delta_m_howel': delta_m_howel, 'beta2_howel': beta2_howel, 'E-Check': abs(beta2_calculated-beta2)}

# LossCoefficient_star

In [34]:
def lc_star(sigma_r, r_m1, r_m2, C_m1, C_m2, N, beta1_star, beta2_star, H1, s_r):
    
    sigma = sigma_r  
    beta1 = beta1_star
    
    U2 = 2*math.pi*r_m2*N/60        #meter/sec

    alpha2 = math.degrees(math.atan(U2/C_m2-math.tan(math.radians(beta2_star))))
    
    W_theta1 = C_m1*math.tan(math.radians(beta1_star))
    W_theta2 = C_m2*math.tan(math.radians(beta2_star))
    
    W1 = C_m1/math.cos(math.radians(beta1_star))
    W2 = C_m2/math.cos(math.radians(beta2_star))
    
    W2_star = W2
    W1_star = W1
    W_m1 = C_m1
    W_m2 = C_m2
    
    r1 = r_m1
    r2 = r_m2
    
    #profile:
    Wmax_W1 = 1.12+0.61*((math.cos(math.radians(beta1_star)))**2)/sigma*((r1*W_theta1-r2*W_theta2)/(r1*W_m1))
    D_eq_star = Wmax_W1*((math.cos(math.radians(beta2_star)))/(math.cos(math.radians(beta1_star))))*(W_m1/W_m2)
    a = 1+3.1*(D_eq_star-1)**2+0.4*(D_eq_star-1)**8
    lc_profile_star = (2*sigma/math.cos(math.radians(beta2_star)))*((W2_star/W1_star)**2)*(0.004*a)
    
    #annulus:
    H = H1
    s = s_r
    
    beta_m_star = math.degrees(math.atan(0.5*(math.tan(math.radians(beta1_star))+math.tan(math.radians(beta2_star)))))
    lc_annulus_star = 0.02*s/H*sigma*((math.cos(math.radians(beta1_star)))**2)/((math.cos(math.radians(beta_m_star)))**3)
    
    #secondary:
    C_D = (lc_profile_star+lc_annulus_star)/(sigma*((math.cos(math.radians(beta1_star)))**2)/((math.cos(math.radians(beta_m_star)))**3))
    C_L = 2/sigma*math.cos(math.radians(beta_m_star))*(math.tan(math.radians(beta1_star))-math.tan(math.radians(beta2_star)))-C_D*math.tan(math.radians(beta_m_star))
    lc_secondary_star = 0.018*(C_L**2)*(sigma*((math.cos(math.radians(beta1_star)))**2)/((math.cos(math.radians(beta_m_star)))**3))
    
    lc_m_star = lc_profile_star+lc_annulus_star+lc_secondary_star
    
    return lc_m_star

# Off-Design

In [2]:
def lc_OD(sigma_r, r_m1, r_m2, C_m1, C_m2, N, beta1, beta2, H1, s_r, M_W1_star, i, blade_type, i_s, i_star, i_c, M_W1, loss_coefficient_star):
   #k_sh:
    k_sh_dic = {
        'C_Series': 1.1,
        '65_Series': 1,
        'DCA': 0.7,
    }
        
    k_sh = k_sh_dic[blade_type[0]] 
    
    if blade_type[0]=='65-Series':
        k = 0.0117
    else:
        k = 0.007
                
    
    if ((M_W1>0.7) & (k_sh<=1)):
        
        R_s = i_s-i_star
        R_c = i_star-i_c
    
        i_c_new = i_star-R_c/(1+0.5*M_W1**3)
    
        i_s_new = i_star+R_c/(1+0.5*(k_sh*M_W1)**3)
    
        i_m = i_c_new+(i_s_new-i_c_new)*R_c/(R_c+R_s)
    
        loss_coefficient_m = loss_coefficient_star*(1+(i_m-i_star)**2/R_s**2)
    
        if i>=i_m:
            loss_coefficient = loss_coefficient_m*(1+((i-i_m)/(i_s_new-i_m))**2)
    
        else:
            loss_coefficient = loss_coefficient_m*(1+((i-i_m)/(i_c_new-i_m))**2)
            
    else:
        
        sigma = sigma_r  
        
        U2 = 2*math.pi*r_m2*N/60        #meter/sec
        alpha2 = math.degrees(math.atan(U2/C_m2-math.tan(math.radians(beta2))))
    
        W_theta1 = C_m1*math.tan(math.radians(beta1))
        W_theta2 = C_m2*math.tan(math.radians(beta2))
    
        W1 = C_m1/math.cos(math.radians(beta1))
        W2 = C_m2/math.cos(math.radians(beta2))

        W_m1 = C_m1
        W_m2 = C_m2
    
        r1 = r_m1
        r2 = r_m2
        
        #profile:
        Wmax_W1 = 1.12+0.61*((math.cos(math.radians(beta1)))**2)/sigma*((r1*W_theta1-r2*W_theta2)/(r1*W_m1))+k*abs(i-i_star)**1.43
        D_eq = Wmax_W1*((math.cos(math.radians(beta2)))/(math.cos(math.radians(beta1))))*(W_m1/W_m2)
        a = 1+3.1*(D_eq-1)**2+0.4*(D_eq-1)**8
        lc_profile = (2*sigma/math.cos(math.radians(beta2)))*((W2/W1)**2)*(0.004*a)        
    
        #annulus:
        H = H1
        s = s_r
    
        beta_m = math.degrees(math.atan(0.5*(math.tan(math.radians(beta1))+math.tan(math.radians(beta2)))))
        lc_annulus = 0.02*s/H*sigma*((math.cos(math.radians(beta1)))**2)/((math.cos(math.radians(beta_m)))**3)
    
        #secondary:
        C_D = (lc_profile+lc_annulus)/(sigma*((math.cos(math.radians(beta1)))**2)/((math.cos(math.radians(beta_m)))**3))
        C_L = 2/sigma*math.cos(math.radians(beta_m))*(math.tan(math.radians(beta1))-math.tan(math.radians(beta2)))-C_D*math.tan(math.radians(beta_m))
        lc_secondary = 0.018*(C_L**2)*(sigma*((math.cos(math.radians(beta1)))**2)/((math.cos(math.radians(beta_m)))**3))
            
        loss_coefficient = lc_profile+lc_annulus+lc_secondary
        
        
    return loss_coefficient

In [37]:
def delta_OD(sigma, beta1, delta_star, i_star, W_m1, W_m2, i):
    
    Ddelta_Di_star = (1+(sigma+0.25*sigma**4)*(beta1/53)**2.5)/(math.e**(3.1*sigma))    
    
    delta = delta_star+Ddelta_Di_star*(i-i_star)+10*(1-(W_m2/W_m1))    
    
    return delta

# Rotor Outlet

In [2]:
def rotor_outlet(M_W1_star, df1, df2, gama, R, blade_type, i_s, i_c, loss_coefficient_star, sigma, beta1, delta_star_r, i_star, i, alpha1, r_m1, r_m2, N, beta_prime2, H1, H2, N_design, t_max_c, a_c, s_r, theta):
    
    C_m1     = df2['C_m1[meter/sec]']['value']
    C1       = df2['C1[meter/sec]']['value']
    U1       = df2['U1[meter/sec]']['value']
    W1       = df2['W1[meter/sec]']['value']
    m_dot    = df2['m_dot[kg/sec]']['value']
    T1       = df1['T[k]']['Point 1']
    P1       = df1['P[bar]']['Point 1']
    P01_rel  = df1['P[bar]']['Point 01_rel']
    h1       = df1['h[kj/kg]']['Point 1']
    h01      = df1['h[kj/kg]']['Point 01']
    C_m2_new = C_m1
    W_m1     = C_m1
    C_m2     = 0
    M_W1     = W1/((gama*R*1000*T1)**0.5)
    W_theta1 = C_m1*math.tan(math.radians(beta1))
    C_theta1 = C_m1*math.tan(math.radians(alpha1))
    
   
    while (abs(C_m2_new-C_m2)>0.001):
        C_m2 = C_m2_new
        W_m2 = C_m2
        
        #delta:
        if (N==N_design)&(i==i_star):
            delta = delta_star(sigma, beta1, t_max_c, blade_type, theta, beta_prime2, a_c, C_m1, C_m2)['delta_m_howel']
            beta2_star = delta+beta_prime2
        else:
            delta = delta_OD(sigma, beta1, delta_star_r, i_star, W_m1, W_m2, i)
            
        #loss_coefficient:
        if (N==N_design)&(i==i_star):
            loss_coefficient =  lc_star(sigma, r_m1, r_m2, C_m1, C_m2, N, beta1, beta2_star, H1, s_r)
        else:
            beta2 = beta_prime2+delta
            loss_coefficient = lc_OD(sigma, r_m1, r_m2, C_m1, C_m2, N, beta1, beta2, H1, s_r, M_W1_star, i, blade_type, i_s, i_star, i_c, M_W1, loss_coefficient_star)

    
        U2 = 2*math.pi*r_m2*N/60        #meter/sec
        beta2 = beta_prime2+delta
        alpha2 = math.degrees(math.atan(U2/C_m2-math.tan(math.radians(beta2))))
    
        C_theta2 = C_m2*math.tan(math.radians(alpha2))
    
        W_theta2 = W_m2*math.tan(math.radians(beta2))
    
        W2 = W_m2/math.cos(math.radians(beta2))
        
        C2 = C_m2/math.cos(math.radians(alpha2))
    
        delta_h0 = (U2*C_theta2-U1*C_theta1)/1000        #kJ/kg   
        
        h2 = h1+0.5*((W1**2)-(U1**2)-(W2**2)+(U2**2))/1000
        #h2 = h1-0.5*((W1**2)-(U1**2)-(W2**2)+(U2**2))/1000
    
        h02_rel = h2+(W2**2)/2000
    
        P02_rel = P01_rel-loss_coefficient*(P01_rel-P1)
    
        #Point02_rel:
        Properties02_rel = properties({'P':P02_rel, 'h':h02_rel}) 
    
        #Point02:
        h02 = h01+delta_h0
        s2 = Properties02_rel['s[kj/kg-k]']
    
        Properties02 = properties({'h':h02, 's':s2}) 
    
        #Point2:
        Properties2 = properties({'h':h2, 's':s2}) 
    
        P2 = Properties2['P[bar]']
        T2 = Properties2['T[k]']
        rho2 = P2*101.325/(R*T2)
    
        A2 = 2*math.pi*r_m2*H2
    
        C_m2_new = m_dot/(rho2*A2)
        #print(C_m2, C_m2_new, W1-W2, h2-h1, M_W1)
        
    
    Properties = {
        'Point 02'     : Properties02,
        'Point 2'      : Properties2,
        'Point 02_rel' : Properties02_rel,
    }
    
    df1 = pd.DataFrame(Properties).T
        
    df2 = pd.DataFrame({
        'm_dot[kg/sec]'   : m_dot,
        'C2[meter/sec]'   : C2,
        'W2[meter/sec]'   : W2,
        'C_m2[meter/sec]' : C_m2,
        'U2[meter/sec]'   : U2,
        'rho2[kg/m^3]'    : rho2,
        'A2[m^2]'         : A2,
        'alpha2[degree]'  : alpha2,
        'beta2[degree]'   : beta2,
        'E-Check'         : abs(C_m2-C_m2_new),  
        
    }, index = ['value'])
    
    
    return df1, df2

# Stator Outlet

In [4]:
def stator_outlet(M_C2_star, df1, df2, gama, R, blade_type, i_s, i_c, loss_coefficient_star, sigma, alpha2, delta_star_s, i_star, i, beta2, r_m2, r_m3, N, alpha_prime3, H2, H3, N_design, t_max_c, a_c, s_s, theta, beta1_star):
    
    C_m2     = df2['C_m2[meter/sec]']['value']
    U2       = df2['U2[meter/sec]']['value']
    C2       = df2['C2[meter/sec]']['value']
    m_dot    = df2['m_dot[kg/sec]']['value']
    T2       = df1['T[k]']['Point 2']
    P2       = df1['P[bar]']['Point 2']
    P02      = df1['P[bar]']['Point 02']
    h2       = df1['h[kj/kg]']['Point 2']
    h02      = df1['h[kj/kg]']['Point 02']
    C_m3_new = C_m2
    W_m2     = C_m2
    C_m3     = 0
    M_C2     = C2/((gama*R*1000*T2)**0.5)
    W_theta2 = C_m2*math.tan(math.radians(beta2))
    C_theta2 = C_m2*math.tan(math.radians(alpha2))
    C3 = 1000
    
    while (abs(C_m3_new-C_m3)>0.0001):
        C_m3 = C_m3_new
        W_m3 = C_m3
        
        #delta:
        if (N==N_design)&(i==i_star):
            delta = delta_star(sigma, beta1_star, t_max_c, blade_type, theta, alpha_prime3, a_c, C_m2, C_m3)['delta_m_howel']
            alpha3_star = delta+alpha_prime3
        else:
            delta = delta_OD(sigma, alpha2, delta_star_s, i_star, W_m2, W_m3, i)
                    
            
        #loss_coefficient:
        if (N==N_design)&(i==i_star):
            loss_coefficient =  lc_star(sigma, r_m2, r_m3, C_m2, C_m3, N, beta1_star, alpha3_star, H2, s_s)
        else:
            alpha3 = alpha_prime3+delta
            loss_coefficient = lc_OD(sigma, r_m2, r_m3, C_m2, C_m3, N, alpha2, alpha3, H2, s_s, M_C2_star, i, blade_type, i_s, i_star, i_c, M_C2, loss_coefficient_star)
                    
        alpha3 = alpha_prime3+delta
    
        C3 = C_m3/math.cos(math.radians(alpha3))
    
        P03 = P02-loss_coefficient*(P02-P2)
        
        h03 = h02        
    
        #Point03:
        Properties03 = properties({'P':P03, 'h':h03}) 
        s3 = Properties03['s[kj/kg-k]']
                
        #Point3:
        h3 = h03-(C3**2)/2000
        Properties3 = properties({'h':h3, 's':s3}) 
        
        P3 = Properties3['P[bar]']
        T3 = Properties3['T[k]']
        rho3 = P3*101.325/(R*T3)
        
    
        A3 = 2*math.pi*r_m3*H3
    
        C_m3_new = m_dot/(rho3*A3)        
        
    
    Properties = {
        'Point 03'     : Properties03,
        'Point 3'      : Properties3,
    }
    
    df1 = pd.DataFrame(Properties).T
        
    df2 = pd.DataFrame({
        'C3[meter/sec]'   : C3,
        'C_m3[meter/sec]' : C_m3,
        'rho3[kg/m^3]'    : rho3,
        'A3[m^2]'         : A3,
        'alpha3[degree]'  : alpha3,
        'E-Check'         : abs(C_m3-C_m3_new),  
        
    }, index = ['value'])
    
    #print(sigma, r_m2, r_m3, C_m2, C_m3, N, beta1_star, alpha3_star, H2, s_s, loss_coefficient)
    
    return df1, df2

# Efficiency

In [41]:
def efficiency(df1_rotor_inlet, df1_rotor_outlet, df1_stator_outlet, df2_rotor_inlet, df2_rotor_outlet, df2_stator_outlet):
    
    #m_dot:
    m_dot = df2_rotor_inlet['m_dot[kg/sec]']['value']
    
    #total enthalpies:
    h01 = df1_rotor_inlet['h[kj/kg]']['Point 01']
    h02 = df1_rotor_outlet['h[kj/kg]']['Point 02']
    h03 = df1_stator_outlet['h[kj/kg]']['Point 03']
    df = pd.DataFrame(pd.array([h01, h02, h03]), index=['1', '2', '3'], columns=['h0[kj/kg]'])
    
    #static enthalpies:
    h1 = df1_rotor_inlet['h[kj/kg]']['Point 1']
    h2 = df1_rotor_outlet['h[kj/kg]']['Point 2']
    h3 = df1_stator_outlet['h[kj/kg]']['Point 3']
    df['h[kj/kg]'] = pd.array([h1, h2, h3])
    
    #relative enthalpies:
    h01_rel = df1_rotor_inlet['h[kj/kg]']['Point 01_rel']
    h02_rel = df1_rotor_outlet['h[kj/kg]']['Point 02_rel']
    df['h0_rel[kj/kg]'] = pd.array([h01_rel, h02_rel, '-'])
    
    #entropies:
    s1 = df1_rotor_inlet['s[kj/kg-k]']['Point 1']
    s2 = df1_rotor_outlet['s[kj/kg-k]']['Point 2']
    s3 = df1_stator_outlet['s[kj/kg-k]']['Point 3']
    df['s[kj/kg-k]'] = pd.array([s1, s2, s3])
    
    #relative total pressures:
    P01_rel = df1_rotor_inlet['P[bar]']['Point 01_rel']
    P02_rel = df1_rotor_outlet['P[bar]']['Point 02_rel']
    df['P0_rel[bar]'] = pd.array([P01_rel, P02_rel, '-'])
    
    #total pressures:
    P01 = df1_rotor_inlet['P[bar]']['Point 01']
    P02 = df1_rotor_outlet['P[bar]']['Point 02']
    P03 = df1_stator_outlet['P[bar]']['Point 03']
    T03 = df1_stator_outlet['T[k]']['Point 03']
    df['P0[bar]'] = pd.array([P01, P02, P03])
    
    #static pressures:
    P1 = df1_rotor_inlet['P[bar]']['Point 1']
    P2 = df1_rotor_outlet['P[bar]']['Point 2']
    P3 = df1_stator_outlet['P[bar]']['Point 3']
    df['P[bar]'] = pd.array([P1, P2, P3])
    
    #absoloute velocities:
    C1 = df2_rotor_inlet['C1[meter/sec]']['value']
    C2 = df2_rotor_outlet['C2[meter/sec]']['value']
    C3 = df2_stator_outlet['C3[meter/sec]']['value']
    df['C[meter/sec]'] = pd.array([C1, C2, C3])
    
    #meridional velocities:
    C_m1 = df2_rotor_inlet['C_m1[meter/sec]']['value']
    C_m2 = df2_rotor_outlet['C_m2[meter/sec]']['value']
    C_m3 = df2_stator_outlet['C_m3[meter/sec]']['value']
    df['C_m[meter/sec]'] = pd.array([C_m1, C_m2, C_m3])
    
    alpha1 = df2_rotor_inlet['alpha1[degree]']['value']
    alpha2 = df2_rotor_outlet['alpha2[degree]']['value']
    alpha3 = df2_stator_outlet['alpha3[degree]']['value']
    
    #tangential velocities:
    df['C_theta[meter/sec]'] = pd.array(
        [
            C_m1*math.tan(math.radians(alpha1)),
            C_m2*math.tan(math.radians(alpha2)),
            C_m3*math.tan(math.radians(alpha3)),
        ]
    )
    
    #relative velocitie:
    W1 = df2_rotor_inlet['W1[meter/sec]']['value']
    W2 = df2_rotor_outlet['W2[meter/sec]']['value']
    df['W[meter/sec]'] = pd.array([W1, W2, '-'])
    
    #absoloute angles:
    df['alpha[degree]'] = pd.array([alpha1, alpha2, alpha3])
    
    #relative angles:
    beta1 = df2_rotor_inlet['beta1[degree]']['value']
    beta2 = df2_rotor_outlet['beta2[degree]']['value']
    df['beta[degree]'] = pd.array([beta1, beta2, '-'])
    
    #densities:
    rho1 = df2_rotor_inlet['rho1[kg/m^3]']['value']
    rho2 = df2_rotor_outlet['rho2[kg/m^3]']['value']
    rho3 = df2_stator_outlet['rho3[kg/m^3]']['value']
    df['rho[kg/m^3]'] = pd.array([rho1, rho2, rho3])
    
    #Point2_s:
    Properties2s = properties({'P':P2, 's':s1}) 
    h2_s = Properties2s['h[kj/kg]']
    
    etta_rotor = (h1-h2_s)/(h1-h2)
    
    #Point3_s:
    Properties3s = properties({'P':P3, 's':s2}) 
    h3_s = Properties3s['h[kj/kg]']
    
    etta_stator = (h2-h3_s)/(h2-h3)
    
    Re = (h2-h1)/(h3-h1)
    
    #Point03ss:
    Properties03ss = properties({'P':P03, 's':s1}) 
    h03ss = Properties03ss['h[kj/kg]']
    
    etta_is_stage = (h03ss-h01)/(h03-h01)
    
    
    dic = {
        'm_dot[kg/sec]'   : m_dot,
        'etta_rotor'     : etta_rotor,
        'etta_stator'    : etta_stator,
        'etta_is_stage'  : etta_is_stage,
        'h01[kj/kg]'     : h01,
        'h2_s[kj/kg]'    : h2_s,
        'h3_s[kj/kg]'    : h3_s,
        'h03ss[kj/kg]'   : h03ss,
        'h03[kj/kg]'     : h03,
        'P03[bar]'       : P03,
        'T03[k]'         : T03,
        'alpha3[degree]' : alpha3,
    }
    
    return df, dic

# Checker Function

In [6]:
def checker(df, dic, i_star_r, i_star_s, delta_star_r, delta_star_s, N_stage):
    
    if abs(i_star_r)<6:
        i_star_r_check = True
    else:
        i_star_r_check = False
        
    if abs(i_star_s)<6:
        i_star_s_check = True
    else:
        i_star_s_check = False
        
    if delta_star_r>0:
        delta_star_r_check = True
    else:
        delta_star_r_check = False
        
    if delta_star_s>0:
        delta_star_s_check = True
    else:
        delta_star_s_check = False
        
    rotor = {'i_star': i_star_r, 'delta_star': delta_star_r}
    rotor_check = {'i_star': i_star_r_check, 'delta_star': delta_star_r_check}
    
    stator = {'i_star': i_star_s, 'delta_star': delta_star_s}
    stator_check = {'i_star': i_star_s_check, 'delta_star': delta_star_s_check}
    
    check = {
        'rotor'        : rotor,
        'rotor_check'  : rotor_check,
        'stator'       : stator,
        'stator_check' : stator_check,
    }
    
    df1 = pd.DataFrame(check)
    
    rotor_check = np.array(df1['rotor_check'])
    stator_check = np.array(df1['stator_check'])
    
    h  = np.array(df['h[kj/kg]'])   
    h0 = np.array(df['h0[kj/kg]']) 
    s  = np.array(df['s[kj/kg-k]'])
    P  = np.array(df['P[bar]'])
    
    
    #if (not h[2] is float) or (not h[2] is float) or (not h[2] is float) or (not h[2] is float):
        #print('yes')
        #flag = 0
        #return df1, flag
    
    check = np.array([
        (np.diff(h) >= 0).all(),
        (np.diff(h0) >= 0).all(),
        (np.diff(s) >= 0).all(),
        (np.diff(P) >= 0).all(),
        (0<dic['etta_rotor']<1),      
        (0<dic['etta_stator']<1),        
        (0<dic['etta_is_stage']<1),
        rotor_check.all(),
        stator_check.all()
    ]
    )
        
    if check.all():
        print(f'everything in stage {N_stage} is alright')
        flag = 1
        
    else:
        flag = 0                                                  #the operating point has no existence
        if not (np.diff(h) >= 0).all():
            print(f'in stage {N_stage}, h is not True')
            
        if not (np.diff(h0) >= 0).all():
            print(f'in stage {N_stage}, h0 is not True')
            
        if not (np.diff(s) >= 0).all():
            print(f'in stage {N_stage}, s is not True')
            
        if not (np.diff(P) >= 0).all():
            print(f'in stage {N_stage}, P is not True')  
            
        if not rotor_check.all():
            print(f'in stage {N_stage}, i_star_rotor = {i_star_r} or delta_star_rotor = {delta_star_r} is not True')   
            
        if not stator_check.all():
            print(f'in stage {N_stage}, i_star_stator = {i_star_s} or delta_star_stator = {delta_star_s} is not True')
    
    return df1, flag

# Stage Solver

In [7]:
def stage_solver_star(alpha1, N_design, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, N_stage):
    #i_star:
    stagger = (beta_prime1+beta_prime2)/2
    theta = beta_prime1-beta_prime2
    
    dic = i_star(beta_prime1, stagger, a_c, theta, t_max_c, sigma_r, blade_type)
    i_star_r = dic['i_star']
    alpha_star = dic['alpha_star']
    beta1_star = dic['beta1_star']
    
    #i_c:
    dic = i_c(i_star_r, alpha_star, beta1_star, theta, stagger)
    i_c_r = dic['i_c']
    alpha_c = dic['alpha_c']
    beta1_c = dic['beta1_c']
    
    #i_s:
    dic = i_s(i_star_r, alpha_star, beta1_star, theta, stagger)
    i_s_r = dic['i_s']
    alpha_s = dic['alpha_s']
    beta1_s = dic['beta1_s']
    
    #rotor inlet:
    i = i_star_r
    N = N_design              #RPM
    sigma = sigma_r
    beta1 = beta_prime1+i
    
    df1_rotor_inlet_star, df2_rotor_inlet_star = rotor_inlet(r_h1, r_t1, N, beta1, alpha1, P01, T01, R)
    
    W1_star = df2_rotor_inlet_star['W1[meter/sec]']['value']
    T1_star = df1_rotor_inlet_star['T[k]']['Point 1']
    M_W1_star = W1_star/((gama*R*1000*T1_star)**0.5)
    
    #delta_star:
    C_m1 = df2_rotor_inlet_star['C_m1[meter/sec]']['value']
    U1 = df2_rotor_inlet_star['U1[meter/sec]']['value']
    h1 = df1_rotor_inlet_star['h[kj/kg]']['Point 1']
    
    df1_rotor_outlet_star, df2_rotor_outlet_star = rotor_outlet(M_W1_star, df1_rotor_inlet_star, df2_rotor_inlet_star, gama, R, blade_type, i_s_r, i_c_r, 0, sigma, beta1, 0, i_star_r, i_star_r, alpha1, r_m1, r_m2, N_design, beta_prime2, H1, H2, N_design, t_max_c, a_c, s_r, theta)
    
    C_m2 = df2_rotor_outlet_star['C_m2[meter/sec]']['value']
        
    dic = delta_star(sigma_r, beta1, t_max_c, blade_type, theta, beta_prime2, a_c, C_m1, C_m2)
    delta_star_r = dic['delta_m_howel']
    beta2_star_r = dic['beta2_howel']
    
    #loss_coefficient_star:
    loss_coefficient_star_r = lc_star(sigma_r, r_m1, r_m2, C_m1, C_m2, N, beta1, beta2_star_r, H1, s_r)
    
    #i_star:
    stagger = (alpha_prime2+alpha_prime3)/2
    theta = alpha_prime2-alpha_prime3
        
    dic = i_star(alpha_prime2, stagger, a_c, theta, t_max_c, sigma_s, blade_type)
    i_star_s = dic['i_star']
    alpha_star = dic['alpha_star']
    beta1_star = dic['beta1_star']
    
    #i_c:
    dic = i_c(i_star_s, alpha_star, beta1_star, theta, stagger)
    i_c_s = dic['i_c']
    alpha_c = dic['alpha_c']
    beta1_c = dic['beta1_c']
    
    #i_s:
    dic = i_s(i_star_r, alpha_star, beta1_star, theta, stagger)
    i_s_s = dic['i_s']
    alpha_s = dic['alpha_s']
    beta1_s = dic['beta1_s']
    
    #delta_star:
    C_m2_star = df2_rotor_outlet_star['C_m2[meter/sec]']['value']
    U2 = 2*math.pi*r_m2*N/60
    C2_star = df2_rotor_outlet_star['C2[meter/sec]']['value']
    T2_star = df1_rotor_outlet_star['T[k]']['Point 2']
    M_C2_star = C2_star/((gama*R*1000*T2_star)**0.5)
    df1_stator_outlet_star, df2_stator_outlet_star = stator_outlet(M_C2_star, df1_rotor_outlet_star, df2_rotor_outlet_star, gama, R, blade_type, i_s_s, i_c_s, 0, sigma_s, beta1_star, 0, i_star_s, i_star_s, math.degrees(math.atan(U2/C_m2_star-math.tan(math.radians(beta1_star)))), r_m2, r_m3, N_design, alpha_prime3, H2, H3, N_design, t_max_c, a_c, s_s, theta, beta1_star)

    C_m3 = df2_stator_outlet_star['C_m3[meter/sec]']['value']
    
    dic = delta_star(sigma_s, beta1_star, t_max_c, blade_type, theta, alpha_prime3, a_c, df2_rotor_outlet_star['C_m2[meter/sec]']['value'], C_m3)
    delta_star_s = dic['delta_m_howel']
    beta2_star_s = dic['beta2_howel']
    
    #lc_star:
    loss_coefficient_star_s = lc_star(sigma_s, r_m2, r_m3, df2_rotor_outlet_star['C_m2[meter/sec]']['value'], C_m3, N_design, beta1_star, beta2_star_s, H2, s_s)
    
    return i_star_r, delta_star_r, beta2_star_r, i_star_s, delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s

In [8]:
def stage_solver(
    alpha1, N_design, P01, T01, beta_prime1, beta_prime2, t_max_c, blade_type, a_c, i, N, sigma_r, sigma_s, alpha_prime2, alpha_prime3,
    r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, N_stage, i_star_r, delta_star_r, beta2_star_r, i_star_s,
    delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s
):
    
    #off_design:
    stagger = (beta_prime1+beta_prime2)/2
    theta = beta_prime1-beta_prime2
    beta1_star = i_star_r+beta_prime1
    sigma = sigma_r
    beta1 = beta_prime1+i
    
    df1_rotor_inlet, df2_rotor_inlet = rotor_inlet(r_h1, r_t1, N, beta1, alpha1, P01, T01, R)
    df1_rotor_outlet, df2_rotor_outlet = rotor_outlet(M_W1_star, df1_rotor_inlet, df2_rotor_inlet, gama, R, blade_type, i_s_r, i_c_r, loss_coefficient_star_r, sigma, beta1, delta_star_r, i_star_r, i, alpha1, r_m1, r_m2, N, beta_prime2, H1, H2, N_design, t_max_c, a_c, s_r, theta)
    
    
    #stator_outlet:
    stagger = (alpha_prime2+alpha_prime3)/2
    theta = alpha_prime2-alpha_prime3
    beta1_star = i_star_s+alpha_prime2
    alpha2 = df2_rotor_outlet['alpha2[degree]']['value']
    C_m2 = df2_rotor_outlet['C_m2[meter/sec]']['value']
    beta2 = df2_rotor_outlet['beta2[degree]']['value']
    i_stator = alpha2-alpha_prime2
    
    #off_design:
    df1_stator_outlet, df2_stator_outlet = stator_outlet(M_C2_star, df1_rotor_outlet, df2_rotor_outlet, gama, R, blade_type, i_s_s, i_c_s, loss_coefficient_star_s, sigma, alpha2, delta_star_s, i_star_s, i_stator, beta2, r_m2, r_m3, N, alpha_prime3, H2, H3, N_design, t_max_c, a_c, s_s, theta, beta1_star)
    
    
    #efficiency:
    df, dic = efficiency(df1_rotor_inlet, df1_rotor_outlet, df1_stator_outlet, df2_rotor_inlet, df2_rotor_outlet, df2_stator_outlet)
    
    
    #checker:
    check, flag = checker(df, dic, i_star_r, i_star_s, delta_star_r, delta_star_s, N_stage)
    
    return df, dic, check, flag

# Compressor Solver

In [2]:
def compressor_solver(compressor_data, gama, i, N, R, N_design, N_stage):
    
    for n in range(1, N_stage+1):
        #first_stage:
        if n==1:
            alpha1, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, r_h1, r_h2, r_t1, r_t2, s_s, s_r =  compressor_data[n-1]
        
            i_star_r, delta_star_r, beta2_star_r, i_star_s, delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s = stage_solver_star(
                alpha1, N_design, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, 
                alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, n
            )
        
            df1, dic1, check1, flag = stage_solver(
                alpha1, N_design, P01, T01, beta_prime1, beta_prime2, t_max_c, blade_type, a_c, i, N, sigma_r, sigma_s, alpha_prime2, alpha_prime3,
                r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, n, i_star_r, delta_star_r, beta2_star_r, i_star_s,
                delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s
            )
            
        
            P03, T03, alpha3 = dic1['P03[bar]'], dic1['T03[k]'], dic1['alpha3[degree]']
        
            h01 = dic1['h01[kj/kg]']
            
            if flag == 0:
                return 0, 0, 0, 0
                break

        
        #middle stages:
        if n>1:
        
            P01, T01 = P03, T03
            alpha1 = alpha3
        
            beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, r_h1, r_h2, r_t1, r_t2, s_s, s_r =  compressor_data[n-1]
        
            i_star_r, delta_star_r, beta2_star_r, i_star_s, delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s = stage_solver_star(
                alpha1, N_design, P01, T01, beta_prime1, beta_prime2, a_c, t_max_c, blade_type, sigma_r, sigma_s, 
                alpha_prime2, alpha_prime3, r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, n
            )
        
            df1, dic1, check1, flag = stage_solver(
                alpha1, N_design, P01, T01, beta_prime1, beta_prime2, t_max_c, blade_type, a_c, i, N, sigma_r, sigma_s, alpha_prime2, alpha_prime3,
                r_m1, r_m2, r_m3, H1, H2, H3, gama, R, r_h1, r_h2, r_t1, r_t2, s_s, s_r, n, i_star_r, delta_star_r, beta2_star_r, i_star_s,
                delta_star_s, beta2_star_s, M_W1_star, M_C2_star, i_s_r, i_c_r, i_s_s, i_c_s, loss_coefficient_star_r, loss_coefficient_star_s
            )
        
            P03, T03, alpha3 = dic1['P03[bar]'], dic1['T03[k]'], dic1['alpha3[degree]']
            
            if flag == 0:
                return 0, 0, 0, 0
                break

        
        if n==N_stage:
            
            if flag == 1:
                print('everything in compressor is ok')
                
            h03_N = dic1['h03[kj/kg]']
            h03_ss_N = dic1['h03ss[kj/kg]']
            m_dot = dic1['m_dot[kg/sec]']
        
            etta_compressor = (h03_ss_N-h01)/(h03_N-h01)
            P01 = compressor_data[0][1]
            P02_P01 = P03/P01
            
    return P02_P01, etta_compressor, m_dot, flag